In [1]:
import pandas as pd

import statsmodels.api as sm
from mlxtend.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# 讀欄位名稱
df = pd.read_csv("./MiningProcess_Flotation_Plant_Database.csv")
columns = df.columns.tolist()
columns

['date',
 '% Iron Feed',
 '% Silica Feed',
 'Starch Flow',
 'Amina Flow',
 'Ore Pulp Flow',
 'Ore Pulp pH',
 'Ore Pulp Density',
 'Flotation Column 01 Air Flow',
 'Flotation Column 02 Air Flow',
 'Flotation Column 03 Air Flow',
 'Flotation Column 04 Air Flow',
 'Flotation Column 05 Air Flow',
 'Flotation Column 06 Air Flow',
 'Flotation Column 07 Air Flow',
 'Flotation Column 01 Level',
 'Flotation Column 02 Level',
 'Flotation Column 03 Level',
 'Flotation Column 04 Level',
 'Flotation Column 05 Level',
 'Flotation Column 06 Level',
 'Flotation Column 07 Level',
 '% Iron Concentrate',
 '% Silica Concentrate']

In [3]:
df_summary = df.describe().T
df_summary.round(2)

,count,unique,top,freq
date,737453,4097,2017-06-16 15:00:00,180
% Iron Feed,737453,278,"64,03",142560
% Silica Feed,737453,293,"6,26",142560
Starch Flow,737453,409317,"2562,5",690
Amina Flow,737453,319416,"534,668",959
Ore Pulp Flow,737453,180189,"402,246",1735
Ore Pulp pH,737453,131143,"10,0591",1509
Ore Pulp Density,737453,105805,"1,75",3214
Flotation Column 01 Air Flow,737453,43675,"299,927",13683
Flotation Column 02 Air Flow,737453,80442,"255,322",1487


In [4]:
missing_values = df.isnull().values.any() 
print(f"missing_values:{missing_values}\n")

missing_values:False



In [5]:
df_1 = df.iloc[:, 1:].map(lambda x: pd.to_numeric(str(x).replace(',', '.'), errors='coerce'))
df_1

,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,Flotation Column 03 Air Flow,...,Flotation Column 07 Air Flow,Flotation Column 01 Level,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate
0,55.20,16.98,3019.53,557.434,395.713,10.06640,1.74000,249.214,253.235,250.576,...,250.884,457.396,432.962,424.954,443.558,502.255,446.370,523.344,66.91,1.31
1,55.20,16.98,3024.41,563.965,397.383,10.06720,1.74000,249.719,250.532,250.862,...,248.994,451.891,429.560,432.939,448.086,496.363,445.922,498.075,66.91,1.31
2,55.20,16.98,3043.46,568.054,399.668,10.06800,1.74000,249.741,247.874,250.313,...,248.071,451.240,468.927,434.610,449.688,484.411,447.826,458.567,66.91,1.31
3,55.20,16.98,3047.36,568.665,397.939,10.06890,1.74000,249.917,254.487,250.049,...,251.147,452.441,458.165,442.865,446.210,471.411,437.690,427.669,66.91,1.31
4,55.20,16.98,3033.69,558.167,400.254,10.06970,1.74000,250.203,252.136,249.895,...,248.928,452.441,452.900,450.523,453.670,462.598,443.682,425.679,66.91,1.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737448,49.75,23.20,2710.94,441.052,386.570,9.62129,1.65365,302.344,298.786,299.163,...,313.695,392.160,430.702,872.008,418.725,497.548,446.357,416.892,64.27,1.71
737449,49.75,23.20,2692.01,473.436,384.939,9.62063,1.65352,303.013,301.879,299.487,...,236.700,401.505,404.616,864.409,418.377,506.398,372.995,426.337,64.27,1.71
737450,49.75,23.20,2692.20,500.488,383.496,9.61874,1.65338,303.662,307.397,299.487,...,225.879,408.899,399.316,867.598,419.531,503.414,336.035,433.130,64.27,1.71
737451,49.75,23.20,1164.12,491.548,384.976,9.61686,1.65324,302.550,301.959,298.045,...,308.115,405.107,466.832,876.591,407.299,502.301,340.844,433.966,64.27,1.71


In [6]:
# Define response variable (y) and independent variables
X = df_1.iloc[:, 1:21] # except data and % iron concentrate
y = df_1.iloc[:, -1]

In [7]:
df_1.to_csv("./reformatted_MiningProcess_Flotation_Plant_Database.csv")

# (a) OLS, stepwise reg

In [8]:
def OLS_fs(X, y, pvalue):

    X = sm.add_constant(X)
    ols_model = sm.OLS(y, X).fit()
    # print(ols_model.summary())

    # Access p-values for each feature
    p_values = ols_model.pvalues
    selected_features = p_values[p_values < pvalue].index

    # Display the selected features
    # print("Selected Features:")
    # print(selected_features)

    return selected_features, ols_model

In [9]:
def stepwise_fs(X, y):
    X = sm.add_constant(X)
    ols_model = sm.OLS(y, X)

    selected_features = []
    for feature in X.columns:
        # Fit the model with the current set of features
        current_features = selected_features + [feature]
        current_X = X[current_features]
        current_model = sm.OLS(y, current_X).fit()

        # Compare the AIC of the current model with the previous model
        if not selected_features or current_model.aic < best_model.aic:
            best_model = current_model
            selected_features.append(feature)

    return selected_features

In [10]:
OLS_selected_features, OLS_model = OLS_fs(X, y, 0.05)
OLS_selected_features.tolist()

['const',
 '% Silica Feed',
 'Starch Flow',
 'Amina Flow',
 'Ore Pulp Flow',
 'Ore Pulp pH',
 'Ore Pulp Density',
 'Flotation Column 01 Air Flow',
 'Flotation Column 02 Air Flow',
 'Flotation Column 03 Air Flow',
 'Flotation Column 04 Air Flow',
 'Flotation Column 05 Air Flow',
 'Flotation Column 06 Air Flow',
 'Flotation Column 01 Level',
 'Flotation Column 02 Level',
 'Flotation Column 03 Level',
 'Flotation Column 04 Level',
 'Flotation Column 05 Level',
 'Flotation Column 06 Level',
 'Flotation Column 07 Level']

In [11]:
stepwise_selected_features = stepwise_fs(X, y)
stepwise_selected_features

['const',
 '% Silica Feed',
 'Starch Flow',
 'Amina Flow',
 'Ore Pulp Flow',
 'Ore Pulp pH',
 'Ore Pulp Density',
 'Flotation Column 01 Air Flow',
 'Flotation Column 02 Air Flow',
 'Flotation Column 03 Air Flow',
 'Flotation Column 04 Air Flow',
 'Flotation Column 05 Air Flow',
 'Flotation Column 06 Air Flow',
 'Flotation Column 01 Level',
 'Flotation Column 02 Level',
 'Flotation Column 03 Level',
 'Flotation Column 04 Level',
 'Flotation Column 05 Level',
 'Flotation Column 07 Level']

# (e) Ridge, Lasso, and Elastic net

In [12]:
def ridge_fs(X, y, alpha):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize the Ridge regression model
    ridge_model = Ridge(alpha=alpha)  
    ridge_model.fit(X_train_scaled, y_train)

    # Record the coefficients of the Ridge model
    coefficients = ridge_model.coef_
    feature_coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})
    selected_features = feature_coefficients[feature_coefficients['Coefficient'] != 0]['Feature']

    return selected_features

In [13]:
def lasso_fs(X, y, alpha):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize the Lasso regression model
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train_scaled, y_train)

    # Record the coefficients of the Lasso model
    coefficients = lasso_model.coef_
    feature_coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})
    selected_features = feature_coefficients[feature_coefficients['Coefficient'] != 0]['Feature']

    return selected_features

In [14]:
def elasticNet_fs(X, y, alpha, l1_ratio):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize the Elastic Net regression model
    elastic_net_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio) 
    elastic_net_model.fit(X_train_scaled, y_train)

    # Record the coefficients of the Elastic Net model
    coefficients = elastic_net_model.coef_
    feature_coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})
    selected_features = feature_coefficients[feature_coefficients['Coefficient'] != 0]['Feature']

    return selected_features

In [15]:
ridge_selected_features = ridge_fs(X,y,1)
ridge_selected_features.tolist()

['% Silica Feed',
 'Starch Flow',
 'Amina Flow',
 'Ore Pulp Flow',
 'Ore Pulp pH',
 'Ore Pulp Density',
 'Flotation Column 01 Air Flow',
 'Flotation Column 02 Air Flow',
 'Flotation Column 03 Air Flow',
 'Flotation Column 04 Air Flow',
 'Flotation Column 05 Air Flow',
 'Flotation Column 06 Air Flow',
 'Flotation Column 07 Air Flow',
 'Flotation Column 01 Level',
 'Flotation Column 02 Level',
 'Flotation Column 03 Level',
 'Flotation Column 04 Level',
 'Flotation Column 05 Level',
 'Flotation Column 06 Level',
 'Flotation Column 07 Level']

In [16]:
lasso_selected_features = lasso_fs(X, y, 0.01)
lasso_selected_features.tolist()

['% Silica Feed',
 'Starch Flow',
 'Amina Flow',
 'Ore Pulp Flow',
 'Ore Pulp pH',
 'Ore Pulp Density',
 'Flotation Column 01 Air Flow',
 'Flotation Column 03 Air Flow',
 'Flotation Column 04 Air Flow',
 'Flotation Column 05 Air Flow',
 'Flotation Column 06 Air Flow',
 'Flotation Column 04 Level',
 'Flotation Column 05 Level',
 'Flotation Column 07 Level']

In [17]:
elasticNet_selected_features = elasticNet_fs(X, y, 0.2, 0.1)
elasticNet_selected_features.tolist()

['% Silica Feed',
 'Starch Flow',
 'Amina Flow',
 'Ore Pulp Flow',
 'Ore Pulp pH',
 'Ore Pulp Density',
 'Flotation Column 01 Air Flow',
 'Flotation Column 02 Air Flow',
 'Flotation Column 03 Air Flow',
 'Flotation Column 04 Air Flow',
 'Flotation Column 05 Air Flow',
 'Flotation Column 06 Air Flow',
 'Flotation Column 04 Level',
 'Flotation Column 05 Level',
 'Flotation Column 06 Level',
 'Flotation Column 07 Level']

# (h) highly correlated columns

In [18]:
# Show highly correlated columns
corr_matrix = df_1.corr().round(5) 
highly_correlated_cols = []
highly_correlated_threshold = 0.8

for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > highly_correlated_threshold:
            highly_correlated_cols.append((corr_matrix.columns[i], corr_matrix.columns[j]))
if len(highly_correlated_cols) > 0:
    print("Highly Correlated Column Pairs:")
    for col1, col2 in highly_correlated_cols:
        print(f"[{col1}, {col2}], corr = {corr_matrix.loc[col1, col2]}")
else:
    print("No highly correlated columns found.")

Highly Correlated Column Pairs:
[% Iron Feed, % Silica Feed], corr = -0.97183
[Flotation Column 01 Air Flow, Flotation Column 02 Air Flow], corr = 0.84828
[Flotation Column 01 Air Flow, Flotation Column 03 Air Flow], corr = 0.95453
[Flotation Column 02 Air Flow, Flotation Column 03 Air Flow], corr = 0.86044
[Flotation Column 06 Air Flow, Flotation Column 07 Air Flow], corr = 0.85407
[% Iron Concentrate, % Silica Concentrate], corr = -0.80056


In [19]:
# Show the table of the coefficient estimation by linear reg
OLS_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     % Silica Concentrate   R-squared:                       0.153
Model:                              OLS   Adj. R-squared:                  0.153
Method:                   Least Squares   F-statistic:                     6655.
Date:                  Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                          23:32:34   Log-Likelihood:            -1.0724e+06
No. Observations:                737453   AIC:                         2.145e+06
Df Residuals:                    737432   BIC:                         2.145e+06
Df Model:                            20                                         
Covariance Type:              nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            4.0365      0.236     17.117      0.000       3.574       4.499
% Silica Feed                    0.0098      0.000     52.012      0.000       0.009       0.010
Starch Flow                   -3.86e-05   1.11e-06    -34.653      0.000   -4.08e-05   -3.64e-05
Amina Flow                       0.0032   1.94e-05    166.360      0.000       0.003       0.003
Ore Pulp Flow                   -0.0047      0.000    -33.818      0.000      -0.005      -0.004
Ore Pulp pH                     -0.3263      0.003    -95.748      0.000      -0.333      -0.320
Ore Pulp Density                -1.9295      0.025    -78.371      0.000      -1.978      -1.881
Flotation Column 01 Air Flow    -0.0089      0.000    -62.878      0.000      -0.009      -0.009
Flotation Column 02 Air Flow     0.0011    8.3e-05     13.184      0.000       0.001       0.001
Flotation Column 03 Air Flow    -0.0055      0.000    -34.928      0.000      -0.006      -0.005
Flotation Column 04 Air Flow     0.0353      0.001     58.274      0.000       0.034       0.037
Flotation Column 05 Air Flow    -0.0065      0.000    -17.872      0.000      -0.007      -0.006
Flotation Column 06 Air Flow     0.0028   8.33e-05     33.629      0.000       0.003       0.003
Flotation Column 07 Air Flow  8.507e-05   8.69e-05      0.979      0.328   -8.53e-05       0.000
Flotation Column 01 Level       -0.0002   1.57e-05    -12.412      0.000      -0.000      -0.000
Flotation Column 02 Level    -4.706e-05   1.44e-05     -3.257      0.001   -7.54e-05   -1.87e-05
Flotation Column 03 Level        0.0002   1.25e-05     15.058      0.000       0.000       0.000
Flotation Column 04 Level       -0.0003   1.89e-05    -15.794      0.000      -0.000      -0.000
Flotation Column 05 Level       -0.0012   2.35e-05    -49.672      0.000      -0.001      -0.001
Flotation Column 06 Level     7.479e-05   1.81e-05      4.135      0.000    3.93e-05       0.000
Flotation Column 07 Level       -0.0003   2.23e-05    -15.400      0.000      -0.000      -0.000
==============================================================================
Omnibus:                    73856.648   Durbin-Watson:                   0.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            98100.819
Skew:                           0.882   Prob(JB):                         0.00
Kurtosis:                       3.285   Cond. No.                     6.74e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.74e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""